In [1]:
import pandas as pd
import hashlib
import re
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /Users/macmini/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/macmini/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
# File paths
paths = {
    'cleaned_for_LLM': "../data/cleaned_for_LLM.csv",
    'cleaned_for_tfidf': "../data/cleaned_for_tfidf.csv", 
    'cleaned_for_bert': "../data/cleaned_for_bert.csv"
}

# Read in cleaned data

In [3]:
# Load CSV
df = pd.read_csv('../data/cleaned_for_modeling.csv')
print(f"Loaded {len(df):,} rows\n")

Loaded 24,172,346 rows



# Process Data

In [4]:
# Define functions

def is_likely_english(text):
    """Fast heuristic: >80% ASCII characters = likely English."""
    if pd.isna(text) or len(str(text).strip()) == 0:
        return False
    text = str(text)
    ascii_count = sum(1 for c in text if ord(c) < 128)
    return (ascii_count / len(text)) > 0.8

def filter_english_only(df):
    """Filter to English-only reviews."""
    print(f"Before English filter: {len(df):,} rows")
    df_clean = df.copy()
    df_clean['is_english'] = df_clean['text'].apply(is_likely_english)
    df_clean = df_clean[df_clean['is_english']].drop(columns=['is_english'])
    print(f"  After: {len(df_clean):,} rows ({len(df) - len(df_clean):,} removed)\n")
    return df_clean

def filter_original_tag(df):
    """Remove reviews with Google's (Original) translation tag."""
    print(f"Before (Original) tag filter: {len(df):,} rows")
    df_clean = df[~df['text'].str.contains(r'\(Original\)', na=False)].copy()
    print(f"  After: {len(df_clean):,} rows ({len(df) - len(df_clean):,} removed)\n")
    return df_clean

In [5]:
# Apply english filtering pipeline

df = (df
    .pipe(filter_english_only)
    .pipe(filter_original_tag)
    .reset_index(drop=True)
)

print(f"Final dataset: {len(df):,} rows")
df.info()

Before English filter: 24,172,346 rows
  After: 24,058,686 rows (113,660 removed)

Before (Original) tag filter: 24,058,686 rows
  After: 22,624,384 rows (1,434,302 removed)

Final dataset: 22,624,384 rows
<class 'pandas.DataFrame'>
RangeIndex: 22624384 entries, 0 to 22624383
Data columns (total 4 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   rating   float64
 2   text     str    
 3   gmap_id  str    
dtypes: float64(2), str(2)
memory usage: 4.0 GB


# Add review_id (unique per review)

In [6]:
# Define review_id generating function
def make_review_id(df):

    # Combine fields into one string
    combined = (
        df["user_id"].astype(str) + "||" +
        df["gmap_id"].astype(str) + "||" +
        df["rating"].astype(str) + "||" +
        df["text"].astype(str)
    )

    # Create SHA256 hash and insert as first column
    df.insert(
        0,
        "review_id",
        combined.apply(lambda x: hashlib.sha256(x.encode("utf-8")).hexdigest())
    )

    return df


In [7]:
# Apply function to dataframe
df = (
    df
    .pipe(make_review_id)
)

In [8]:
df.head(10)

,review_id,user_id,rating,text,gmap_id
0,456e420929727f933dbaed63eff45cde53c7b92438cf0d...,1.067134e+20,5.0,"Easy process, extremely friendly, helpful staf...",0x80960c29f2e3bf29:0x4b291f0d275a5699
1,ea2ad448a8b443c1c42c5d4ca9dd84d02fe9f2f110b993...,1.024963e+20,5.0,My girlfriends and I took a weekend ski trip t...,0x80960c29f2e3bf29:0x4b291f0d275a5699
2,77efbe6a6f4d27512b59bb2f878b0ac8b533aa03a11fb7...,1.102407e+20,5.0,The team at Black Tie never disappoints our se...,0x80960c29f2e3bf29:0x4b291f0d275a5699
3,ba742a26b57396fde7a05136bbbf551906f5c6d9e66008...,1.116481e+20,5.0,They were awesome people! First timer they hel...,0x80960c29f2e3bf29:0x4b291f0d275a5699
4,57787d7d11fc5e75d7a7643fd966534de03fdc72cf764f...,1.169394e+20,5.0,"Great service, fast, responsive, they came and...",0x80960c29f2e3bf29:0x4b291f0d275a5699
5,6cd920c382945084ed2537452eef5d2603e3c19e4c26d6...,1.035190e+20,5.0,Awesome! Incredible customer service. Profess...,0x80960c29f2e3bf29:0x4b291f0d275a5699
6,275023c6626a913042ec003365906cbf377ca45cfc0acf...,1.021517e+20,5.0,"Unforgettable, remarkable, customer for life. ...",0x80960c29f2e3bf29:0x4b291f0d275a5699
7,1bc8c890f35919ec310e65b64701a07a249989f0adcdd3...,1.110766e+20,5.0,"Good service, affordable",0x80960c29f2e3bf29:0x4b291f0d275a5699
8,56a745e1bb00e7475f21f1fdd5051b5d328c54c74747ef...,1.089911e+20,5.0,"Nice people, good service.",0x80960c29f2e3bf29:0x4b291f0d275a5699
9,32d6a0dcb40c36f484a2dfc47b50ae9b6eb86b0cc143dd...,1.070499e+20,5.0,Awesome!!!!!,0x80960c29f2e3bf29:0x4b291f0d275a5699


In [9]:
# Drop duplicates (found 5 when exploring the data)
before = len(df)
df = df.drop_duplicates(subset="review_id", keep="first")
after = len(df)

print(f"Rows before deduplication: {before:,}")
print(f"Rows after deduplication:  {after:,}")
print(f"Duplicates removed:        {before - after:,}")

Rows before deduplication: 22,624,384
Rows after deduplication:  22,624,379
Duplicates removed:        5


## Aspect Extraction (Weak Supervision)

In [10]:
aspect_keywords = {
    'food_quality': ['food', 'meal', 'dish', 'taste', 'flavor', 'delicious', 'fresh',
                     'menu', 'order', 'coffee', 'drink', 'burger', 'chicken', 'pizza'],
    'service': ['service', 'staff', 'employee', 'manager', 'worker', 'associate',
                'waiter', 'waitress', 'server', 'cashier', 'friendly', 'rude',
                'helpful', 'unprofessional', 'told', 'said', 'asked'],
    'wait_time': ['wait', 'waiting', 'slow', 'fast', 'quick', 'minutes', 'hour',
                  'line', 'queue', 'busy', 'long', 'forever', 'delay', 'prompt'],
    'price_value': ['price', 'prices', 'expensive', 'cheap', 'cost', 'value', 'worth',
                    'affordable', 'overpriced', 'reasonable', 'pricey', 'money', 'deal'],
    'cleanliness': ['clean', 'dirty', 'filthy', 'hygiene', 'gross', 'spotless',
                    'mess', 'messy', 'tidy', 'maintained'],
    'atmosphere': ['atmosphere', 'ambiance', 'decor', 'seating', 'crowded', 'quiet',
                   'noisy', 'comfortable', 'cozy', 'parking', 'location', 'space']
}

aspects = list(aspect_keywords.keys())
sentiments = ['positive', 'negative', 'neutral']
label_cols = [f'{aspect}_{sentiment}' for aspect in aspects for sentiment in sentiments]

# Weak supervision: assign sentiment based on star rating (ratings 1-2 = negative, 3 = neutral, 4-5 = positive)
def get_sentiment(rating):
    if rating <= 2:
        return 'negative'
    elif rating >= 4:
        return 'positive'
    else:
        return 'neutral'

# For each aspect, detect keyword presence in review text and assign
# the review-level sentiment label, producing 18 binary aspect-sentiment columns
print("Extracting aspect labels...")
print()
df['sentiment'] = df['rating'].map(get_sentiment)

for aspect, keywords in aspect_keywords.items():
    pattern = '|'.join(keywords)
    aspect_mentioned = df['text'].str.lower().str.contains(pattern, na=False)
    
    for sentiment in sentiments:
        df[f'{aspect}_{sentiment}'] = ((aspect_mentioned) & (df['sentiment'] == sentiment)).astype(float)

df = df.drop(columns=['sentiment'])

# Group by aspect
print("Label distribution:")
for aspect in aspects:
    print(f"\n{aspect}:")
    for sentiment in sentiments:
        col = f'{aspect}_{sentiment}'
        print(f"  {sentiment:<10} {df[col].sum():>12,}")

Extracting aspect labels...

Label distribution:

food_quality:
  positive    6,522,008.0
  negative      829,286.0
  neutral       706,123.0

service:
  positive    5,578,686.0
  negative      948,162.0
  neutral       522,983.0

wait_time:
  positive    2,808,088.0
  negative      638,891.0
  neutral       422,151.0

price_value:
  positive    2,550,283.0
  negative      434,621.0
  neutral       363,091.0

cleanliness:
  positive    1,152,334.0
  negative      205,862.0
  neutral       148,413.0

atmosphere:
  positive    1,858,548.0
  negative      203,707.0
  neutral       237,925.0


In [11]:
df.head(1)

,review_id,user_id,rating,text,gmap_id,food_quality_positive,food_quality_negative,food_quality_neutral,service_positive,service_negative,...,wait_time_neutral,price_value_positive,price_value_negative,price_value_neutral,cleanliness_positive,cleanliness_negative,cleanliness_neutral,atmosphere_positive,atmosphere_negative,atmosphere_neutral
0,456e420929727f933dbaed63eff45cde53c7b92438cf0d...,1.067134e+20,5.0,"Easy process, extremely friendly, helpful staf...",0x80960c29f2e3bf29:0x4b291f0d275a5699,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Save data
df.to_csv(paths['cleaned_for_LLM'], index=False)
print(f"Saved to {paths['cleaned_for_LLM']}\n")
df.info()

Saved to ../data/cleaned_for_LLM.csv

<class 'pandas.DataFrame'>
Index: 22624379 entries, 0 to 22624383
Data columns (total 23 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   review_id              str    
 1   user_id                float64
 2   rating                 float64
 3   text                   str    
 4   gmap_id                str    
 5   food_quality_positive  float64
 6   food_quality_negative  float64
 7   food_quality_neutral   float64
 8   service_positive       float64
 9   service_negative       float64
 10  service_neutral        float64
 11  wait_time_positive     float64
 12  wait_time_negative     float64
 13  wait_time_neutral      float64
 14  price_value_positive   float64
 15  price_value_negative   float64
 16  price_value_neutral    float64
 17  cleanliness_positive   float64
 18  cleanliness_negative   float64
 19  cleanliness_neutral    float64
 20  atmosphere_positive    float64
 21  atmosphere_negative    float64

# For Advanced Model
Large Language Model-based ABSA using prompting
* None, wants raw text, capitalization, punctuation, etc.

In [11]:
# Load CSV (for LLM, bring this code to model pipeline)
df = pd.read_csv(
    paths["cleaned_for_LLM"],
    usecols=["review_id", "rating", "gmap_id", "text"
            'food_quality_positive', 'food_quality_negative', 'food_quality_neutral',
            'service_positive', 'service_negative', 'service_neutral',
            'wait_time_positive', 'wait_time_negative', 'wait_time_neutral',
            'price_value_positive', 'price_value_negative', 'price_value_neutral',
            'cleanliness_positive', 'cleanliness_negative', 'cleanliness_neutral',
            'atmosphere_positive', 'atmosphere_negative', 'atmosphere_neutral'
            ],
    dtype={
        "review_id": "string",
        "rating": "int8",
        "gmap_id": "string",
        "text": "string",
    },
    nrows=20
)
df.head(10)

,review_id,rating,text,gmap_id
0,456e420929727f933dbaed63eff45cde53c7b92438cf0d...,5,"Easy process, extremely friendly, helpful staf...",0x80960c29f2e3bf29:0x4b291f0d275a5699
1,ea2ad448a8b443c1c42c5d4ca9dd84d02fe9f2f110b993...,5,My girlfriends and I took a weekend ski trip t...,0x80960c29f2e3bf29:0x4b291f0d275a5699
2,77efbe6a6f4d27512b59bb2f878b0ac8b533aa03a11fb7...,5,The team at Black Tie never disappoints our se...,0x80960c29f2e3bf29:0x4b291f0d275a5699
3,ba742a26b57396fde7a05136bbbf551906f5c6d9e66008...,5,They were awesome people! First timer they hel...,0x80960c29f2e3bf29:0x4b291f0d275a5699
4,57787d7d11fc5e75d7a7643fd966534de03fdc72cf764f...,5,"Great service, fast, responsive, they came and...",0x80960c29f2e3bf29:0x4b291f0d275a5699
5,6cd920c382945084ed2537452eef5d2603e3c19e4c26d6...,5,Awesome! Incredible customer service. Profess...,0x80960c29f2e3bf29:0x4b291f0d275a5699
6,275023c6626a913042ec003365906cbf377ca45cfc0acf...,5,"Unforgettable, remarkable, customer for life. ...",0x80960c29f2e3bf29:0x4b291f0d275a5699
7,1bc8c890f35919ec310e65b64701a07a249989f0adcdd3...,5,"Good service, affordable",0x80960c29f2e3bf29:0x4b291f0d275a5699
8,56a745e1bb00e7475f21f1fdd5051b5d328c54c74747ef...,5,"Nice people, good service.",0x80960c29f2e3bf29:0x4b291f0d275a5699
9,32d6a0dcb40c36f484a2dfc47b50ae9b6eb86b0cc143dd...,5,Awesome!!!!!,0x80960c29f2e3bf29:0x4b291f0d275a5699


# For Baseline Model
Logistic regression or Naive Bayes + TF-IDF
* Lowercasing 
* Remove punctuation
* Remove stop words, except for negations like not/no (in model pipeline)
* N-grams (bigrams) (in model pipeline)
* Lemmatization (maybe, if want to boost model)
* Sentence-level tokens (maybe, if want to boost model)

In [13]:
def baseline_text_chunked_csv(in_path: str, out_path: str, chunksize: int, text_col: str, new_col: str):
    """
    Memory-safe baseline text processing
    Reads `in_path` in chunks, creates `new_col` by:
      - lowercasing
      - removing punctuation (keeping letters/numbers/spaces)
      - normalizing whitespace
      - stripping leading/trailing spaces

    Writes the result incrementally to `out_path`.

    Parameters
    ----------
    in_path : str
        Input CSV path (e.g., "reviews_with_review_id.csv")
    out_path : str
        Output CSV path (e.g., "reviews_with_text_classical.csv")
    chunksize : int
        Number of rows per chunk. Lower if you still crash (e.g., 20_000).
    text_col : str
        Name of the raw text column in the input CSV.
    new_col : str
        Name of the output processed text column.
    """
    first = True

    # Use keep_default_na=False so empty strings don't become NaN unexpectedly
    reader = pd.read_csv(in_path, chunksize=chunksize, keep_default_na=False)

    for i, chunk in enumerate(reader, start=1):
        if text_col not in chunk.columns:
            raise KeyError(
                f"Column '{text_col}' not found. Available columns: {list(chunk.columns)}"
            )

        s = chunk[text_col].astype("string")

        chunk[new_col] = (
            s.str.lower()
             .str.replace(r"[^a-z0-9\s]", " ", regex=True)
             .str.replace(r"\s+", " ", regex=True)
             .str.strip()
        )

        # Write chunk to output CSV (append after first chunk)
        chunk.to_csv(out_path, mode="w" if first else "a", index=False, header=first)
        first = False

        # Lightweight progress
        print(f"✅ Processed chunk {i:,} (rows: {len(chunk):,})")

    print(f"🎉 Done. Wrote cleaned CSV to: {out_path}")


In [14]:
baseline_text_chunked_csv(
    in_path=paths["cleaned_for_LLM"],
    out_path=paths["cleaned_for_tfidf"],
    chunksize=500_000,
    text_col="text",
    new_col="text_baseline",
)

✅ Processed chunk 1 (rows: 500,000)
✅ Processed chunk 2 (rows: 500,000)
✅ Processed chunk 3 (rows: 500,000)
✅ Processed chunk 4 (rows: 500,000)
✅ Processed chunk 5 (rows: 500,000)
✅ Processed chunk 6 (rows: 500,000)
✅ Processed chunk 7 (rows: 500,000)
✅ Processed chunk 8 (rows: 500,000)
✅ Processed chunk 9 (rows: 500,000)
✅ Processed chunk 10 (rows: 500,000)
✅ Processed chunk 11 (rows: 500,000)
✅ Processed chunk 12 (rows: 500,000)
✅ Processed chunk 13 (rows: 500,000)
✅ Processed chunk 14 (rows: 500,000)
✅ Processed chunk 15 (rows: 500,000)
✅ Processed chunk 16 (rows: 500,000)
✅ Processed chunk 17 (rows: 500,000)
✅ Processed chunk 18 (rows: 500,000)
✅ Processed chunk 19 (rows: 500,000)
✅ Processed chunk 20 (rows: 500,000)
✅ Processed chunk 21 (rows: 500,000)
✅ Processed chunk 22 (rows: 500,000)
✅ Processed chunk 23 (rows: 500,000)
✅ Processed chunk 24 (rows: 500,000)
✅ Processed chunk 25 (rows: 500,000)
✅ Processed chunk 26 (rows: 500,000)
✅ Processed chunk 27 (rows: 500,000)
✅ Processe

In [14]:
# Load CSV (for tfidf, bring this code to model pipeline)
df = pd.read_csv(
    paths["cleaned_for_tfidf"],
    usecols=["review_id", "rating", "gmap_id", "text"
            'food_quality_positive', 'food_quality_negative', 'food_quality_neutral',
            'service_positive', 'service_negative', 'service_neutral',
            'wait_time_positive', 'wait_time_negative', 'wait_time_neutral',
            'price_value_positive', 'price_value_negative', 'price_value_neutral',
            'cleanliness_positive', 'cleanliness_negative', 'cleanliness_neutral',
            'atmosphere_positive', 'atmosphere_negative', 'atmosphere_neutral'
            ],
    dtype={
        "review_id": "string",
        "rating": "int8",
        "text_baseline": "string",
        "gmap_id": "string",
   },
   nrows=20
)
df.head(10)

,review_id,rating,gmap_id,text_baseline
0,456e420929727f933dbaed63eff45cde53c7b92438cf0d...,5,0x80960c29f2e3bf29:0x4b291f0d275a5699,easy process extremely friendly helpful staff ...
1,ea2ad448a8b443c1c42c5d4ca9dd84d02fe9f2f110b993...,5,0x80960c29f2e3bf29:0x4b291f0d275a5699,my girlfriends and i took a weekend ski trip t...
2,77efbe6a6f4d27512b59bb2f878b0ac8b533aa03a11fb7...,5,0x80960c29f2e3bf29:0x4b291f0d275a5699,the team at black tie never disappoints our se...
3,ba742a26b57396fde7a05136bbbf551906f5c6d9e66008...,5,0x80960c29f2e3bf29:0x4b291f0d275a5699,they were awesome people first timer they help...
4,57787d7d11fc5e75d7a7643fd966534de03fdc72cf764f...,5,0x80960c29f2e3bf29:0x4b291f0d275a5699,great service fast responsive they came and pi...
5,6cd920c382945084ed2537452eef5d2603e3c19e4c26d6...,5,0x80960c29f2e3bf29:0x4b291f0d275a5699,awesome incredible customer service profession...
6,275023c6626a913042ec003365906cbf377ca45cfc0acf...,5,0x80960c29f2e3bf29:0x4b291f0d275a5699,unforgettable remarkable customer for life i j...
7,1bc8c890f35919ec310e65b64701a07a249989f0adcdd3...,5,0x80960c29f2e3bf29:0x4b291f0d275a5699,good service affordable
8,56a745e1bb00e7475f21f1fdd5051b5d328c54c74747ef...,5,0x80960c29f2e3bf29:0x4b291f0d275a5699,nice people good service
9,32d6a0dcb40c36f484a2dfc47b50ae9b6eb86b0cc143dd...,5,0x80960c29f2e3bf29:0x4b291f0d275a5699,awesome


# For Primary Model
Transformer-based embeddings (e.g., distilBERT) + simple classifier (e.g., logistic regression)
* Sentence-level tokens

In [15]:
def reviews_to_sentences_chunked_csv(
    in_path: str, 
    out_path: str, 
    chunksize: int,
    id_col: str = "review_id", 
    text_col: str = "text", 
    gmap_col: str = "gmap_id",
    rating_col: str = "rating",
    min_chars: int = 3,
):
    """
    Memory-safe sentence splitting for very large review CSVs.

    Reads reviews in chunks → splits into sentences → writes a new CSV
    with columns:
        review_id, gmap_id, rating, sentence_id, sentence_text
    """

    first = True

    reader = pd.read_csv(
        in_path,
        chunksize=chunksize,
        usecols=[id_col, gmap_col, rating_col, text_col],
        keep_default_na=False,
    )

    for i, chunk in enumerate(reader, start=1):
        rows = []

        for rid, txt, gid, rating in zip(
            chunk[id_col].astype(str),
            chunk[text_col].astype(str),
            chunk[gmap_col].astype(str),
            chunk[rating_col],
        ):
            # Minimal normalization only (BERT-safe)
            txt = " ".join(txt.split()).strip()
            if not txt:
                continue

            for j, sent in enumerate(sent_tokenize(txt)):
                sent = sent.strip()
                if len(sent) >= min_chars:
                    rows.append((rid, gid, rating, j, sent))

        sent_df = pd.DataFrame(
            rows,
            columns=[id_col, gmap_col, rating_col, "sentence_id", "sentence_text"],
        )

        # Write incrementally to CSV
        sent_df.to_csv(out_path, mode="w" if first else "a", index=False, header=first)
        first = False

        print(f"✅ Processed chunk {i:,} | sentences written: {len(sent_df):,}")

    print(f"🎉 Finished. Sentence-level CSV saved to:\n{out_path}")

In [16]:
reviews_to_sentences_chunked_csv(
    in_path=paths["cleaned_for_LLM"],
    out_path=paths["cleaned_for_bert"],
    chunksize=250_000,
)

✅ Processed chunk 1 | sentences written: 705,587
✅ Processed chunk 2 | sentences written: 670,275
✅ Processed chunk 3 | sentences written: 615,794
✅ Processed chunk 4 | sentences written: 576,244
✅ Processed chunk 5 | sentences written: 552,183
✅ Processed chunk 6 | sentences written: 546,189
✅ Processed chunk 7 | sentences written: 534,669
✅ Processed chunk 8 | sentences written: 625,251
✅ Processed chunk 9 | sentences written: 623,727
✅ Processed chunk 10 | sentences written: 609,713
✅ Processed chunk 11 | sentences written: 601,281
✅ Processed chunk 12 | sentences written: 599,025
✅ Processed chunk 13 | sentences written: 589,696
✅ Processed chunk 14 | sentences written: 589,201
✅ Processed chunk 15 | sentences written: 582,623
✅ Processed chunk 16 | sentences written: 579,820
✅ Processed chunk 17 | sentences written: 583,657
✅ Processed chunk 18 | sentences written: 560,725
✅ Processed chunk 19 | sentences written: 556,053
✅ Processed chunk 20 | sentences written: 547,589
✅ Process

In [17]:
# Load CSV (for bert, bring this code to model pipeline)
df = pd.read_csv(
    paths["cleaned_for_bert"],
    usecols=["review_id", "rating", "gmap_id", "sentence_id", "sentence_text"],
    dtype={
        "review_id": "string",
        "rating": "int8",
        "gmap_id": "string",
        "sentence_id": "int",
        "sentence_text": "string",
    },
    nrows=20
)
df.head(20)

,review_id,gmap_id,rating,sentence_id,sentence_text
0,456e420929727f933dbaed63eff45cde53c7b92438cf0d...,0x80960c29f2e3bf29:0x4b291f0d275a5699,5,0,"Easy process, extremely friendly, helpful staff."
1,456e420929727f933dbaed63eff45cde53c7b92438cf0d...,0x80960c29f2e3bf29:0x4b291f0d275a5699,5,1,In and out with three kids in 20mins.
2,ea2ad448a8b443c1c42c5d4ca9dd84d02fe9f2f110b993...,0x80960c29f2e3bf29:0x4b291f0d275a5699,5,0,My girlfriends and I took a weekend ski trip t...
3,ea2ad448a8b443c1c42c5d4ca9dd84d02fe9f2f110b993...,0x80960c29f2e3bf29:0x4b291f0d275a5699,5,1,The service and prices were amazing!
4,ea2ad448a8b443c1c42c5d4ca9dd84d02fe9f2f110b993...,0x80960c29f2e3bf29:0x4b291f0d275a5699,5,2,They even came to our house to change out skie...
5,ea2ad448a8b443c1c42c5d4ca9dd84d02fe9f2f110b993...,0x80960c29f2e3bf29:0x4b291f0d275a5699,5,3,The staff is knowledgeable and friendly.
6,ea2ad448a8b443c1c42c5d4ca9dd84d02fe9f2f110b993...,0x80960c29f2e3bf29:0x4b291f0d275a5699,5,4,I will definitely rent from here next time!
7,77efbe6a6f4d27512b59bb2f878b0ac8b533aa03a11fb7...,0x80960c29f2e3bf29:0x4b291f0d275a5699,5,0,The team at Black Tie never disappoints our se...
8,77efbe6a6f4d27512b59bb2f878b0ac8b533aa03a11fb7...,0x80960c29f2e3bf29:0x4b291f0d275a5699,5,1,It’s clear that the service integrity is spot-...
9,77efbe6a6f4d27512b59bb2f878b0ac8b533aa03a11fb7...,0x80960c29f2e3bf29:0x4b291f0d275a5699,5,2,We always use the service located at Sierra Ne...
